In [38]:
import re 
import numpy as np
import pandas as pd 
import matplotlib as mp 


Danish CPR numbers consist of a six-digit date (DDMMYY), followed by a 4 digit identifier (IIII). Using the re module, the task is to write and apply a regular expression that matches any CPR number, either in the DDMMYYIIII format, or in the DDMMYY-IIII format. The regular expression should contain four groups, such that the DD, MM, YY, and IIII parts can be extracted after matching.


In [126]:
text = '''adakaf
310702-7121
dad, 1001011110,fksfjwokff., 31314
222276-4390
1315208121
	'''
#A pattern that matches CPR in DDMMYYIIII format
# 
pattern= re.compile(r'(\d\d)(\d\d)(\d\d)-?(\d{4})')


In [130]:
match= pattern.finditer(text)

for x in match : 
    print(x.groups())
    dd = x.group(1)
    mm= x.group(2)
    yy =x.group(3)
    zz= x.group(4)
    print (yy)
    if '0'<=yy<='99' and '1'<=zz<='3999' :
        print ('Born in 1900')

    elif  '0'<=yy<='36' and '4000'<=zz<='4999' :
        print ('Born in 2000')

    elif  '37'<=yy<='99' and '4000'<=zz<='4999' :
        print ('Born in 1900')

    elif  '00'<=yy<='57' and '5000'<=zz<='8999' :
        print ('Born in 2000')

    elif  '58'<=yy<='99' and '5000'<=zz<='8999' :
        print ('Born in 1800')
    
    elif  '00'<yy<'36' and '9000'<zz<'9999' :
        print ('Born in 2000')

    elif  '37'<yy<'99' and '9000'<zz<'9999' :
        print ('Born in 1900')

('31', '07', '02', '7121')
02
Born in 2000
('10', '01', '01', '1110')
01
Born in 1900
('22', '22', '76', '4390')
76
Born in 1900
('13', '15', '20', '8121')
20
Born in 2000
